In [1]:
import pandas as pd
import numpy as np
import json
import re

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
#importing the dataset
with open('datasets/obiwanintents.json') as file:
    data = json.load(file)
    
# parse through data
responses={}
inputs = []
labels = []
label_map = {}

count = 0
for intent in data['intents']:
    responses[intent['tag']]=intent['responses']
    label_map[intent['tag']] = count
    count += 1
    for message in intent['inputs']:
        inputs.append(message)
        labels.append(intent['tag'])
    
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "labels":labels})


# shuffle the data
data = data.sample(frac=1).reset_index(drop=True)

data['labels'].replace(label_map, inplace=True)

# standardize data to be lowercase and no punctuation
def standardize(string):
    return re.sub(r'[^\w\s]', '', string).lower()

for i in range(len(data)):
    data['inputs'].iloc[i] = standardize(data['inputs'].iloc[i])

# reverse map for finding tag
label_map = dict((v, k) for k, v in label_map.items())
data

C:\Users\matta\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,inputs,labels
0,who is mace windu,7
1,whats your best story,11
2,i can beat you to a pulp,12
3,is mace windu the most powerful jedi,7
4,is general grevious bad,8
...,...,...
155,has the objective been completed,6
156,is ki adi mundi the most powerful jedi,7
157,are you able to help me,5
158,i am looking for help,5


In [3]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(data['inputs'])
X = tokenizer.texts_to_sequences(data['inputs'])
X = pad_sequences(X)

X.shape

(160, 12)

In [4]:
# create model
embedding_dim = 50
input_shape = X.shape[1]

inputs = keras.Input(input_shape, name='input')

lyr = keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=embedding_dim, name='embedding')(inputs)
lyr = keras.layers.LSTM(embedding_dim, return_sequences=True, name='lstm')(lyr)
lyr = keras.layers.Flatten(name='flatten')(lyr)
# lyr = keras.layers.Dense(600, activation='relu', name='dense')(lyr)

outputs = keras.layers.Dense(13, activation='softmax', name='output')(lyr)

model = keras.Model(inputs=inputs, outputs=outputs, name='ObiWan_Kenobot')
model.summary()

Model: "ObiWan_Kenobot"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 12)]              0         
                                                                 
 embedding (Embedding)       (None, 12, 50)            10150     
                                                                 
 lstm (LSTM)                 (None, 12, 50)            20200     
                                                                 
 flatten (Flatten)           (None, 600)               0         
                                                                 
 output (Dense)              (None, 13)                7813      
                                                                 
Total params: 38,163
Trainable params: 38,163
Non-trainable params: 0
_________________________________________________________________


In [5]:
# compiling the model
# sparse categorical cross entropy loss since each caategory is mutually exclusive
model.compile(loss="sparse_categorical_crossentropy",
              optimizer='adam', metrics=['accuracy'])

# training the model
history = model.fit(X, data['labels'], 
                    epochs=10000, 
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, monitor='loss')])

Epoch 1/10000
5/5 [==============================] - 3s 14ms/step - loss: 2.5656 - accuracy: 0.0500
Epoch 2/10000
5/5 [==============================] - 0s 17ms/step - loss: 2.5527 - accuracy: 0.1312
Epoch 3/10000
5/5 [==============================] - 0s 17ms/step - loss: 2.5392 - accuracy: 0.2500
Epoch 4/10000
5/5 [==============================] - 0s 18ms/step - loss: 2.5215 - accuracy: 0.2625
Epoch 5/10000
5/5 [==============================] - 0s 14ms/step - loss: 2.4960 - accuracy: 0.2250
Epoch 6/10000
5/5 [==============================] - 0s 12ms/step - loss: 2.4805 - accuracy: 0.2062
Epoch 7/10000
5/5 [==============================] - 0s 11ms/step - loss: 2.4553 - accuracy: 0.2188
Epoch 8/10000
5/5 [==============================] - 0s 11ms/step - loss: 2.4263 - accuracy: 0.2188
Epoch 9/10000
5/5 [==============================] - 0s 10ms/step - loss: 2.3903 - accuracy: 0.2250
Epoch 10/10000
5/5 [==============================] - 0s 11ms/step - loss: 2.3390 - accuracy: 0.2250

5/5 [==============================] - 0s 7ms/step - loss: 0.0601 - accuracy: 1.0000
Epoch 83/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0568 - accuracy: 1.0000
Epoch 84/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0546 - accuracy: 1.0000
Epoch 85/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0525 - accuracy: 1.0000
Epoch 86/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0514 - accuracy: 1.0000
Epoch 87/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0494 - accuracy: 1.0000
Epoch 88/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0470 - accuracy: 1.0000
Epoch 89/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0475 - accuracy: 1.0000
Epoch 90/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0449 - accuracy: 1.0000
Epoch 91/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0450 - accuracy: 1.0000
Epoch 92/10000


5/5 [==============================] - 0s 8ms/step - loss: 0.0096 - accuracy: 1.0000
Epoch 164/10000
5/5 [==============================] - 0s 8ms/step - loss: 0.0096 - accuracy: 1.0000
Epoch 165/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 166/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0093 - accuracy: 1.0000
Epoch 167/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 168/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 169/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0089 - accuracy: 1.0000
Epoch 170/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0087 - accuracy: 1.0000
Epoch 171/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0086 - accuracy: 1.0000
Epoch 172/10000
5/5 [==============================] - 0s 8ms/step - loss: 0.0086 - accuracy: 1.0000
Epoch 

5/5 [==============================] - 0s 7ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 245/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 246/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 247/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 248/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 249/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 250/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 251/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 252/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 253/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 

5/5 [==============================] - 0s 7ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 326/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 327/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 328/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 329/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 330/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 331/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 332/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 333/10000
5/5 [==============================] - 0s 7ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 334/10000
5/5 [==============================] - 0s 6ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 

...

5/5 [==============================] - 0s 8ms/step - loss: 1.5341e-06 - accuracy: 1.0000
Epoch 2648/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.5333e-06 - accuracy: 1.0000
Epoch 2649/10000
5/5 [==============================] - 0s 8ms/step - loss: 1.5423e-06 - accuracy: 1.0000
Epoch 2650/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.5281e-06 - accuracy: 1.0000
Epoch 2651/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.5222e-06 - accuracy: 1.0000
Epoch 2652/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.5214e-06 - accuracy: 1.0000
Epoch 2653/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.5072e-06 - accuracy: 1.0000
Epoch 2654/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.5058e-06 - accuracy: 1.0000
Epoch 2655/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.5087e-06 - accuracy: 1.0000
Epoch 2656/10000
5/5 [==============================] - 0s 7ms/

5/5 [==============================] - 0s 6ms/step - loss: 1.2629e-06 - accuracy: 1.0000
Epoch 2725/10000
5/5 [==============================] - 0s 6ms/step - loss: 1.2599e-06 - accuracy: 1.0000
Epoch 2726/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.2591e-06 - accuracy: 1.0000
Epoch 2727/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.2606e-06 - accuracy: 1.0000
Epoch 2728/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.2554e-06 - accuracy: 1.0000
Epoch 2729/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.2502e-06 - accuracy: 1.0000
Epoch 2730/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.2435e-06 - accuracy: 1.0000
Epoch 2731/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.2390e-06 - accuracy: 1.0000
Epoch 2732/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.2442e-06 - accuracy: 1.0000
Epoch 2733/10000
5/5 [==============================] - 0s 6ms/

5/5 [==============================] - 0s 7ms/step - loss: 1.0408e-06 - accuracy: 1.0000
Epoch 2802/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.0416e-06 - accuracy: 1.0000
Epoch 2803/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.0349e-06 - accuracy: 1.0000
Epoch 2804/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.0341e-06 - accuracy: 1.0000
Epoch 2805/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.0304e-06 - accuracy: 1.0000
Epoch 2806/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.0297e-06 - accuracy: 1.0000
Epoch 2807/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.0312e-06 - accuracy: 1.0000
Epoch 2808/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.0289e-06 - accuracy: 1.0000
Epoch 2809/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.0267e-06 - accuracy: 1.0000
Epoch 2810/10000
5/5 [==============================] - 0s 7ms/

5/5 [==============================] - 0s 8ms/step - loss: 8.6054e-07 - accuracy: 1.0000
Epoch 2879/10000
5/5 [==============================] - 0s 7ms/step - loss: 8.5309e-07 - accuracy: 1.0000
Epoch 2880/10000
5/5 [==============================] - 0s 7ms/step - loss: 8.5533e-07 - accuracy: 1.0000
Epoch 2881/10000
5/5 [==============================] - 0s 7ms/step - loss: 8.5533e-07 - accuracy: 1.0000
Epoch 2882/10000
5/5 [==============================] - 0s 7ms/step - loss: 8.4937e-07 - accuracy: 1.0000
Epoch 2883/10000
5/5 [==============================] - 0s 7ms/step - loss: 8.4489e-07 - accuracy: 1.0000
Epoch 2884/10000
5/5 [==============================] - 0s 8ms/step - loss: 8.4489e-07 - accuracy: 1.0000
Epoch 2885/10000
5/5 [==============================] - 0s 8ms/step - loss: 8.4340e-07 - accuracy: 1.0000
Epoch 2886/10000
5/5 [==============================] - 0s 8ms/step - loss: 8.3744e-07 - accuracy: 1.0000
Epoch 2887/10000
5/5 [==============================] - 0s 7ms/

5/5 [==============================] - 0s 7ms/step - loss: 7.0557e-07 - accuracy: 1.0000
Epoch 2956/10000
5/5 [==============================] - 0s 7ms/step - loss: 7.0184e-07 - accuracy: 1.0000
Epoch 2957/10000
5/5 [==============================] - 0s 7ms/step - loss: 7.1227e-07 - accuracy: 1.0000
Epoch 2958/10000
5/5 [==============================] - 0s 7ms/step - loss: 6.9588e-07 - accuracy: 1.0000
Epoch 2959/10000
5/5 [==============================] - 0s 8ms/step - loss: 6.9216e-07 - accuracy: 1.0000
Epoch 2960/10000
5/5 [==============================] - 0s 7ms/step - loss: 6.9290e-07 - accuracy: 1.0000
Epoch 2961/10000
5/5 [==============================] - 0s 7ms/step - loss: 6.9439e-07 - accuracy: 1.0000
Epoch 2962/10000
5/5 [==============================] - 0s 7ms/step - loss: 6.9216e-07 - accuracy: 1.0000
Epoch 2963/10000
5/5 [==============================] - 0s 7ms/step - loss: 6.9365e-07 - accuracy: 1.0000
Epoch 2964/10000
5/5 [==============================] - 0s 7ms/

In [6]:
greeting = 'how are you'
goodbye = 'See you later.'
thanks = 'Thank you so much!'
tasks = 'What can you do?'
alive = 'Are you even alive at all?'
hlp = 'PLease I need your help!'
mission = 'Can you tell me a little bit about your mission?'
jedi = 'are you a jedi?'
sith = 'Who is the most evil sith in the galaxy?'
bounty = 'Do you know of Jango Fett?'
funny = 'I know you know some good jokes'
stories = 'Cmon, tell me a story!'
threat = 'I will fight you'

messages = [greeting, goodbye, thanks, tasks, alive, hlp, mission, jedi, sith, bounty, funny, stories, threat]

for msg in messages:
    msg = standardize(msg)
    X = tokenizer.texts_to_sequences([msg])
    X = [item for i in X for item in i]
    X = pad_sequences([X], input_shape)
    pred = model.predict(X)
    pred = pred.argmax()
    print(label_map[pred])
    print()

greeting

goodbye

thanks

tasks

alive

help

mission

alive

jedi

bounty hounter

funny

stories

threat

